# Frame Overlap Tutorial (v0.2.0)

This tutorial demonstrates the new object-oriented API introduced in version 0.2.0 for analyzing neutron Time-of-Flight frame overlap data.

## Overview

The frame_overlap package provides four main classes:

1. **Data**: Load and process ToF data with convolution, frame overlap, and Poisson sampling
2. **Reconstruct**: Apply deconvolution filters to reconstruct original signals
3. **Model**: Simplified nbragg integration for transmission fitting
4. **Analysis**: Fit reconstructed data to extract material parameters

## Correct Workflow Order:

```
Data → Convolute → Overlap → Poisson → Reconstruct
          ↓           ↓         ↓          ↓
      Instrument  Frame ops   Noise   Recover signal
       response              added    before overlap
```

**Key Point**: The reconstruction recovers the **convolved** signal (before overlap), not the raw data. This makes sense because:
- Convolution = instrument response (can't be undone)
- Overlap = mathematical operation (can be undone via deconvolution)

## New Features in v0.2.0:
- **Two-subplot plotting**: Data + residuals in sigma units  
- **Enhanced customization**: Separate kwargs for data and residual plots  
- **nbragg compatibility**: Direct integration via `recon.table`  
- **Model class**: Predefined cross-sections  
- **Better chi² formatting**: 2 sig figs or scientific notation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from frame_overlap import Data, Reconstruct, Analysis, ParametricScan, Model

# Set up plotting
# plt.style.use('default')
%matplotlib inline

## 1. Data Loading and Processing

First, let's load neutron ToF data. The `Data` class handles both signal and openbeam files.

In [ ]:
# Load data from CSV files
data = Data(signal_file='iron_powder.csv',
            openbeam_file='openbeam.csv',
            flux=1e6,           # neutrons/cm²/s
            duration=1.0,       # hours
            freq=20,            # Hz
            threshold=0)        # filter stacks < threshold

print(data)
print(f"\nLoaded {len(data.table)} data points")
print(f"Time range: {data.table['time'].min():.1f} - {data.table['time'].max():.1f} µs")

### Visualize the raw data

In [ ]:
# Plot signal
fig = data.plot()
plt.show()


## 2. Instrument Response Convolution

Simulate the effect of measuring with an instrument that has a finite pulse duration.

In [ ]:
# Apply square response convolution (200 µs pulse)
data.convolute_response(pulse_duration=200)

print("Convolution applied")
print(f"Data shape: {data.table.shape}")

In [ ]:
data.plot(show_stages=True,show_errors=False);

## 3. Frame Overlap Creation

Create overlapping frames with a custom time sequence. The sequence `[0, 12, 10, 25]` means:
- Frame 1 starts at t = 0 ms
- Frame 2 starts at t = 12 ms
- Frame 3 starts at t = 12 + 10 = 22 ms
- Frame 4 starts at t = 22 + 25 = 47 ms

In [ ]:
# Create frame overlap using 'kernel' parameter
# Use a simple 2-frame overlap for this example
data.overlap(kernel=[0, 25], total_time=50)

print(f"Frame overlap created with kernel: {data.kernel}")
print(f"  Frame 1 starts at: 0 ms")
print(f"  Frame 2 starts at: 25 ms")
print(f"New data length: {len(data.table)}")

# Plot overlapped data
fig = data.plot(kind="transmission", show_errors=False)
plt.title('Data after Frame Overlap')
plt.show()

In [ ]:
data.kernel

## 4. Poisson Sampling

Apply Poisson counting statistics to simulate realistic measurement conditions.

In [ ]:
# Apply Poisson sampling AFTER overlap
# You can use duty_cycle or instrument parameters
data.poisson_sample(flux=1e6, measurement_time=1.0, freq=20)

# Plot final processed data
fig = data.plot(kind="transmission", show_errors=False)
plt.title('Data after Poisson Sampling (final overlapped+noisy data)')
plt.show()

## 5. Signal Reconstruction

The `Reconstruct` class applies deconvolution to recover the convolved signal (before overlap).

**What we're recovering**: The signal after convolution but BEFORE overlap (this is stored as `reference_data`)

In [ ]:
data.kernel

In [ ]:
# Create Reconstruct object
recon = Reconstruct(data)

# Apply Wiener filter to recover convolved signal
# noise_power controls regularization (higher = more smoothing)
recon.filter(kind='wiener', noise_power=0.01)

print("Reconstruction complete!")
print(f"\nWhat we recovered: Convolved signal (before overlap)")
print(f"Reference shape: {recon.reference_data.shape}")
print(f"Reconstructed shape: {recon.reconstructed_data.shape}")

print(f"\nReconstruction Statistics:")
for key, value in recon.get_statistics().items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

### Visualize Reconstruction Results

The new two-subplot layout shows:
- **Top**: Convolved (target) vs Reconstructed transmission
- **Bottom**: Residuals in sigma units: (reconstructed - target) / target_error

In [ ]:
# Default transmission plot with residuals
fig = recon.plot(kind='transmission', show_errors=True)
plt.show()

# Can also plot signal counts instead of transmission
fig = recon.plot(kind='signal', show_errors=True)
plt.show()

In [ ]:
# The Model class provides simplified nbragg integration
try:
    model = Model(xs='iron_square_response', vary_weights=True, vary_background=True)
    print("Model created successfully!")
    print(model)
    print("\nTo fit:")
    print("  result = model.fit(recon)")
    print("  result.plot()")
except ImportError:
    print("nbragg not installed")
    print("Install with: pip install nbragg")

In [ ]:
model = Model(xs='iron_square_response', vary_weights=True, vary_background=True)
model

### NEW: Simplified Model Class

The new `Model` class provides an easy interface to nbragg with predefined cross-sections!

In [ ]:
# Check that recon.table is available for nbragg
print("nbragg compatibility check:")
print(f"  recon.table is available: {recon.table is not None}")
print(f"  recon.table shape: {recon.table.shape}")
print(f"  recon.table columns: {list(recon.table.columns)}")
print(f"\nFirst few rows:")
print(recon.table.head())

# Now you can use recon directly with nbragg:
# import nbragg
# xs = nbragg.CrossSection(iron=nbragg.materials["Fe_sg225_Iron-gamma"])
# result = nbragg.TransmissionModel(xs).fit(recon)  # Works!

### NEW: Advanced Plot Customization

Separate kwargs for data and residual plots using the `residual_` prefix!

In [ ]:
# Create Analysis object
analysis = Analysis(recon)

# Set material composition (default is Fe-alpha + 4% Cellulose)
analysis.set_cross_section(['Fe_alpha', 'Cellulose'], [0.96, 0.04])

# Fit with square response function
try:
    analysis.fit(response='square')
    
    # Print fit report
    print(analysis.get_fit_report())
except Exception as e:
    print(f"Fitting failed: {e}")
    print("This can happen with random test data")
analysis

### Visualize fit results

In [ ]:
# Complete workflow in a chained style
data_chain = (Data(signal_file='iron_powder.csv', threshold=30)
              .convolute_response(pulse_duration=200)
              .overlap(kernel=[0, 12, 10, 25])  # Note: 'kernel' parameter (was 'seq')
              .poisson_sample(duty_cycle=0.8))

print("Data processing complete via method chaining")
print(data_chain)

# Continue with reconstruction
recon_chain = Reconstruct(data_chain).filter(kind='wiener', noise_power=0.01)

# Format chi2 nicely
chi2_val = recon_chain.statistics.get('chi2_per_dof', None)
chi2_formatted = recon_chain._format_chi2(chi2_val)
print(f"\nReconstruction χ²/dof: {chi2_formatted}")

## Summary

This tutorial covered:

1. **Data Loading**: Loading signal and openbeam ToF data with proper units (µs internally, ms for display)
2. **Convolution**: Applying instrument response functions (pulse_duration in µs)
3. **Frame Overlap**: Creating overlapping frame sequences using `kernel` parameter
4. **Poisson Sampling**: Simulating counting statistics (duty_cycle or instrument parameters)
5. **Reconstruction**: Deconvolving signals with multiple methods (Wiener, Lucy-Richardson, Tikhonov)
6. **NEW: Enhanced Plotting**: Two-subplot layout with data + residuals in sigma units
7. **NEW: Plot Customization**: Separate kwargs for data and residual plots
8. **NEW: nbragg Integration**: Direct compatibility via `recon.table` property
9. **NEW: Model Class**: Simplified fitting with predefined cross-sections
10. **Analysis**: Fitting for material parameters (legacy Analysis class)
11. **Parametric Scans**: Exploring parameter sensitivity
12. **Method Chaining**: Concise workflow syntax

## Key API Changes in v0.2.0:

- `seq` → `kernel` (overlap method parameter)
- `reconstructed_table` → `reconstructed_data` (consistent naming)
- `reference_table` → `reference_data` (consistent naming)
- New `recon.table` property for nbragg compatibility
- New `Model` class for simplified nbragg integration
- Plot residuals are now in sigma units: `(reconstructed - poisson) / poisson_err`
- Chi-squared formatted with 2 sig figs or scientific notation
- Time stored in microseconds, displayed in milliseconds

## Predefined Model Cross-Sections:

- `'iron_with_cellulose'`: Iron with cellulose background
- `'iron_square_response'`: Iron with square response function

The new v0.2.0 API provides a clean, intuitive interface for neutron ToF frame overlap analysis with physicist-friendly features!

## Additional Resources

- Documentation: See README.md for complete API reference
- Legacy API: The v0.1.0 functional API remains available for backward compatibility
- Source code: https://github.com/TsvikiHirsh/frame_overlap
- nbragg package: https://github.com/neutronimaging/nbragg

In [ ]:
# Method chaining example with CORRECT order
data_chain = (Data(signal_file='iron_powder.csv')
              .convolute_response(pulse_duration=200)
              .overlap(kernel=[0, 25])        # Overlap BEFORE Poisson!
              .poisson_sample(duty_cycle=0.5))

print("Data processing complete via method chaining")
print(data_chain)

# Reconstruct
recon_chain = Reconstruct(data_chain).filter(kind='wiener', noise_power=0.01)
chi2_formatted = recon_chain._format_chi2(recon_chain.statistics.get('chi2_per_dof'))
print(f"\nReconstruction χ²/dof: {chi2_formatted}")

## Summary

This tutorial demonstrated the complete workflow:

**Processing Order**: Data → Convolute → Overlap → Poisson → Reconstruct

**Key Points**:
1. **Convolution** simulates instrument response (stays in signal)
2. **Overlap** creates overlapped frames (what we want to undo)
3. **Poisson** adds counting noise (comes AFTER overlap)
4. **Reconstruction** recovers convolved signal (before overlap)

**New v0.2.0 Features**:
- Two-subplot plotting with residuals in σ units
- Separate kwargs: `ylim` for data, `residual_ylim` for residuals
- Chi² formatting: 2 sig figs or scientific notation
- nbragg integration via `recon.table`
- Model class with predefined cross-sections

**API Changes**:
- `seq` → `kernel` parameter
- `reconstructed_table` → `reconstructed_data`
- Reference is now `convolved_data` (target to reconstruct)

In [ ]:
try:
    # Parameter sensitivity plot
    fig = scan.plot_parameter_sensitivity('pulse_duration', 'recon_r_squared', groupby='n_frames')
    plt.show()
    
    # 2D heatmap
    fig = scan.plot_heatmap('pulse_duration', 'n_frames', 'fit_thickness')
    plt.show()
    
    # Summary statistics
    fig = scan.plot_summary()
    plt.show()
except Exception as e:
    print(f"Plotting failed: {e}")

## 8. Method Chaining Example

The new API supports fluent method chaining for concise workflows.

In [ ]:
# Complete workflow in a chained style
data_chain = (Data(signal_file='iron_powder.csv', threshold=30)
              .convolute_response(pulse_duration=200)
              .overlap(seq=[0, 12, 10, 25])
              .poisson_sample(duty_cycle=0.8))

print("Data processing complete via method chaining")
print(data_chain)

# Continue with reconstruction
recon_chain = Reconstruct(data_chain).filter(kind='wiener', noise_power=0.01)
print(f"\nReconstruction χ²/dof: {recon_chain.statistics.get('chi2_per_dof', 'N/A')}")

## Summary

This tutorial covered:

1. **Data Loading**: Loading signal and openbeam ToF data
2. **Convolution**: Applying instrument response functions
3. **Frame Overlap**: Creating overlapping frame sequences
4. **Poisson Sampling**: Simulating counting statistics
5. **Reconstruction**: Deconvolving signals with multiple methods
6. **Analysis**: Fitting for material parameters
7. **Parametric Scans**: Exploring parameter sensitivity
8. **Method Chaining**: Concise workflow syntax

The new v0.2.0 API provides a clean, intuitive interface for neutron ToF frame overlap analysis!

## Additional Resources

- Documentation: See README.md for complete API reference
- Legacy API: The v0.1.0 functional API remains available for backward compatibility
- Source code: https://github.com/TsvikiHirsh/frame_overlap